## Ablation Analysis

In [1]:
import warnings
import os
import sys
import gc
import warnings

In [ ]:
import anndata as ad
import scanpy as sc
import copy
import torch
from pathlib import Path
import networkx as nx
from sklearn.neighbors import kneighbors_graph
import numpy as np
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.stats
import tqdm

sys.path.append("/home/icb/kemal.inecik/work/codes/tardis")
import tardis
from tardis._utils.functions import create_random_mask
import sklearn
import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
tardis.config = tardis.config_server

In [ ]:
torch.cuda.is_available()

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
_rcparams_path = "/home/icb/kemal.inecik/work/codes/tardis/training/local/figures/rcparams.pickle"
with open(_rcparams_path, 'rb') as file:
    _rcparams = pickle.load(file)
plt.rcParams.update(_rcparams)

In [ ]:
# ablation_adata_path = os.path.join(tardis.config.io_directories['processed'], "figures", "ablation_afriat.h5ad")
ablation_data_path = os.path.join(tardis.config.io_directories['processed'], "figures",  "ablation_afriat.pickle")

In [ ]:
# Load the data from the pickle file
with open(ablation_data_path, 'rb') as file:
    data_loaded = pickle.load(file)

# Access your data
n_leiden_df = data_loaded['n_leiden_df']
latent_dict = data_loaded['latent_dict']
reconst_r2_all = data_loaded['reconst_r2_all']
reconst_r2_valid = data_loaded['reconst_r2_valid']
mi_normalized = data_loaded['mi_normalized']
max_mig_all = data_loaded['max_mig_all']
max_mig_valid = data_loaded['max_mig_valid']

adata = ad.read_h5ad(ablation_adata_path)

In [ ]:
max_mig_valid

In [ ]:
max_mig_all

In [ ]:
n_leiden_df

In [ ]:
# a metric to calculate the number of clusters e.g. it should be 3 in sample status etc.
# aws? knn etc?